In [2]:
# 產生日期範圍

import datetime

def dt_to_str(date_obj):
    return '{:04}'.format(date_obj.year)+'-'+'{:02}'.format(date_obj.month)+'-'+'{:02}'.format(date_obj.day)

def str_to_dt(string):
    word = string.split('-')
    y = int(word[0])
    m = int(word[1])
    d = int(word[2])
    return datetime.date(y, m, d)

start = datetime.date(2022, 5, 5)
end = datetime.date(2022, 5, 26)
time = start
lst = []

donnot_get = ['2021-06-01', '2021-07-15', '2021-08-05', '2021-08-23', '2021-09-15', '2021-10-08', '2021-10-31', '2021-11-20', '2021-12-12']

while time <= end:
    if time not in [str_to_dt(i) for i in donnot_get]:
        lst += [time]
    time += datetime.timedelta(1)

#print(lst)

words = []

for t in lst:
    if words == []:
        words += [[dt_to_str(t)]]
    elif len(words[-1]) == 7:
        words += [[dt_to_str(t)]]
    else:
        words[-1] += [dt_to_str(t)]

#pprint(words)



In [4]:
# 抓取資料

import json
import csv
import requests
import datetime
from tqdm.notebook import tqdm
import time

def str_to_dt(string):
    from datetime import date
    word = string.split('-')
    y = int(word[0])
    m = int(word[1])
    d = int(word[2])
    return date(y, m, d)

with open('ptx.py') as f:
    exec(f.read())
app_id = ''
app_key = ''
a = Auth(app_id, app_key)



for ver, tq in zip(words, tqdm(words)):
    #ver = ['2022-04-29', '2022-04-30',]
    ver_word = ','.join(ver)
    response = requests.get(f'https://ptx.transportdata.tw:443/MOTCAdvanced/v2/Historical/Rail/TRA/LiveTrainDelay?Dates={ver_word}&%24format=CSV', headers= a.get_auth_header())
    #response = requests.get(f'https://ptx.transportdata.tw:443/MOTCAdvanced/v2/Historical/Rail/TRA/DailyTrainInfo/Date/{ver_word}?%24format=CSV', headers= a.get_auth_header())

    content = response.content.decode('utf8')

    csv_file = list(csv.reader(content.splitlines()))

    if len(csv_file) <= 1:
        print('No Data in: ',ver_word)
        time.sleep(1.1)
        continue

    title = [i.encode('utf-8').decode('utf-8-sig') for i in csv_file[0]]   #encode和decode的運用方式還不清楚，但這個方法是目前唯一能解決問題的方法

    for v in ver:
        i = 1
        with open(f'.\\DailyDelayInfo\\{v}.csv', 'w', newline='',encoding='utf-8-sig') as f:
            wrt = csv.writer(f)
            wrt.writerow(title)
            if csv_file[i] == []:
                break
            data_amt = 0
            while i < len(csv_file):
                if datetime.strptime(csv_file[i][5], "%Y-%m-%d %H:%M:%S%z").date() == str_to_dt(v):
                    data_amt += 1
                    wrt.writerow(csv_file[i])
                i += 1
            #if data_amt == 0:
                #print('No Data in', v)
    time.sleep(1.1)

print('Done')


  0%|          | 0/4 [00:00<?, ?it/s]

Done
